# Document used
https://shalabhsingh.github.io/Text-Generation-Word-Predict/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

# important concepts


* Backpropogation through time


* Vanashing Gradient and exponential Gradient

While training using BPTT the gradients have to travel from the last cell all the way to the first cell. The product of these gradients can go to zero or increase exponentially. The exploding gradients problem refers to the large increase in the norm of the gradient during training. The vanishing gradients problem refers to the opposite behavior, when long term components go exponentially fast to norm 0, making it impossible for the model to learn correlation between temporally distant events.

Initializing the weights to identity matrices and biases to zero help prevent the weights from shrinking to zero


# Solving problem statement

predict the gdp 

General steps to follow:

    Initialize weight matrices U, V, W from random distribution and bias b, c with zeros
        Forward propagation to compute predictions
        Compute the loss
        Back-propagation to compute gradients
        Update weights based on gradients
    Repeat steps 2–5

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import numpy as np
np.random.seed(21)

import matplotlib.pyplot as plt
import pandas as pd
import math
import requests

import tensorflow as tf
tf.random.set_seed(21)

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, GRU, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import watermark

%load_ext watermark
%watermark -n -v -m -p numpy,scipy,keras,matplotlib,sklearn,pandas

In [3]:
dataset = pd.read_csv('data/GDP.csv', header=0, parse_dates=[0], index_col=0)
plt.plot(dataset['GDP'])
plt.xlabel('Date')
plt.ylabel('GDP');
plt.gcf().set_size_inches(11,8)
plt.show()
dataset.tail()

FileNotFoundError: [Errno 2] File data/GDP.csv does not exist: 'data/GDP.csv'

In [ ]:
series = dataset['GDP'].pct_change().dropna().values.reshape(-1,1)
series=np.squeeze(series)
#print(np.squeeze(series).shape)
dataset.loc[1:, 'diff']=series

In [ ]:
values = series.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
values = scaler.fit_transform(values)
X = values[:-1]
y = values[1:]

n_points = len(series)
train_points = int(2/3*n_points)+1
X_train, X_test = X[:train_points], X[train_points:]
y_train, y_test = y[:train_points], y[train_points:]
print("We have", len(X_train), "training points and", X_test.shape[0], "testing points")

In [ ]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
#tf.reset_default_graph()
keras.backend.clear_session()

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
print(X_train.shape)
print(X_test.shape)

# model training

In [ ]:
in_shape=(X_train.shape[1], X_train.shape[2])
model = Sequential()
model.add(SimpleRNN(units=10, input_shape=in_shape))
#model.add(LSTM(units=10, input_shape=(X2_train.shape[1], X2_train.shape[2])))
#model.add(GRU(4, input_shape=in_shape))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

In [ ]:
nb_epoch = 50

model.fit(X_train, y_train, epochs=nb_epoch, batch_size=16)

## Prediction

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

plt.plot(range(len(y_train)), y_train)
plt.plot(range(len(y_train_pred)), y_train_pred)



In [ ]:
dataset['RNN_train_predict'] = 1
dataset['RNN_test_predict'] = 1
dataset.loc[1:train_points+1, 'RNN_train_predict'] = 1+scaler.inverse_transform(y_train_pred)
dataset.loc[train_points+2:, 'RNN_test_predict'] = 1+scaler.inverse_transform(y_test_pred)

dataset['RNN_train_predict'] = dataset.iloc[0]['GDP']*dataset['RNN_train_predict'].cumprod()
dataset['RNN_test_predict'] = dataset.iloc[train_points+1]['GDP']*dataset['RNN_test_predict'].cumprod()

dataset[['GDP', 'RNN_train_predict', 'RNN_test_predict']].plot()
plt.gcf().set_size_inches(11, 8)

